# Project Part 2
## Question 1
### Imports and data wrangling


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.optimize import brentq


In [11]:
irs = pd.read_excel("IR Data.xlsx", 
                sheet_name = 'Swaption',
                header=2)
fra = pd.read_csv('fra.csv')
ois = pd.read_csv('ois.csv')

In [13]:
irs.head()

,Expiry,Tenor,-200bps,-150bps,-100bps,-50bps,-25bps,ATM,+25bps,+50bps,+100bps,+150bps,+200bps
0,1Y,1Y,91.57,62.03,44.130,31.224,26.182,22.50,20.96,21.40,24.34,27.488,30.297
1,1Y,2Y,83.27,61.24,46.570,35.807,31.712,28.72,27.12,26.84,28.51,31.025,33.523
2,1Y,3Y,73.92,56.87,44.770,35.745,32.317,29.78,28.29,27.80,28.77,30.725,32.833
3,1Y,5Y,55.19,44.64,36.510,30.242,27.851,26.07,24.98,24.56,25.12,26.536,28.165
4,1Y,10Y,41.18,35.04,30.207,26.619,25.351,24.47,23.98,23.82,24.25,25.204,26.355


In [14]:
fra.head()

,start,tenor,f_irs,f_ois
0,1,1,0.031922,0.003495
1,1,2,0.033217,0.003520
2,1,3,0.033982,0.003662
3,1,5,0.035237,0.003997
4,1,10,0.038406,0.004734


In [15]:
ois.head()

,years,Rate,df,Tenor,Product,f,cumsum_df
0,0.5,0.00250,0.998752,6m,OIS,0.002498,0.998752
1,1.0,0.00300,0.997009,1y,OIS,0.003493,1.995761
2,1.5,NaN,0.995270,NaN,NaN,NaN,2.991030
3,2.0,0.00325,0.993531,2y,OIS,0.003495,3.984561
4,2.5,NaN,0.991773,NaN,NaN,NaN,4.976334


In [58]:
bps = [-.0200, -.0150, -.0100, -.0050, -.0025, 0, .0025, .0050, .0100, .0150, .0200]
df_strike = irs.copy()
for i,forward_rate in enumerate(fra.f_irs):
    for j, bp in enumerate(bps):
        df_strike.iloc[i,j+2] = forward_rate + bp

df_strike.head()

,Expiry,Tenor,-200bps,-150bps,-100bps,-50bps,-25bps,ATM,+25bps,+50bps,+100bps,+150bps,+200bps
0,1Y,1Y,0.011922,0.016922,0.021922,0.026922,0.029422,0.031922,0.034422,0.036922,0.041922,0.046922,0.051922
1,1Y,2Y,0.013217,0.018217,0.023217,0.028217,0.030717,0.033217,0.035717,0.038217,0.043217,0.048217,0.053217
2,1Y,3Y,0.013982,0.018982,0.023982,0.028982,0.031482,0.033982,0.036482,0.038982,0.043982,0.048982,0.053982
3,1Y,5Y,0.015237,0.020237,0.025237,0.030237,0.032737,0.035237,0.037737,0.040237,0.045237,0.050237,0.055237
4,1Y,10Y,0.018406,0.023406,0.028406,0.033406,0.035906,0.038406,0.040906,0.043406,0.048406,0.053406,0.058406


# Displaced-Diffusion Model
Assumes a linear combination of Geometric and Arithmetic Brownian Process to model swap rates

In [23]:
# ATM Black IV
irs.head(2)

,Expiry,Tenor,-200bps,-150bps,-100bps,-50bps,-25bps,ATM,+25bps,+50bps,+100bps,+150bps,+200bps
0,1Y,1Y,91.57,62.03,44.13,31.224,26.182,22.50,20.96,21.40,24.34,27.488,30.297
1,1Y,2Y,83.27,61.24,46.57,35.807,31.712,28.72,27.12,26.84,28.51,31.025,33.523


In [ ]:
# Calculate PVBP
def pvbp(t, T):
    """
    Return PVBP for a period from t to T in years t should be >= 0.5
    assumed to be annual payments
    """

    # TODO ask prof if it is discounted by OIS and if semi-annual or annual payments

    return ois.loc[(ois.years > t) & (ois.years <= T)]


pvbp(5,7)

,years,Rate,df,Tenor,Product,f,cumsum_df
10,5.5,NaN,0.980959,NaN,NaN,NaN,10.887825
11,6.0,NaN,0.979733,NaN,NaN,NaN,11.867558
12,6.5,NaN,0.978508,NaN,NaN,NaN,12.846067
13,7.0,0.004,0.977283,7y,OIS,0.005003,13.823349
